# Erstellung der Rohdaten für die Vorverarbeitung
- Extraktion der Merkmale aus den Ladekurven (Leistung und Ladezeit)
- Zusammenfassen der Ladevorgänge mit den extrahierten Merkmalen


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot
import matplotlib.dates
from datetime import datetime

In [ ]:
# Extraktion der Leistungsgrößen und der Ladezeit aus allen Ladekurven.  

path = r'Datenpfad\\'

daten = []

for file in [f for f in os.listdir(path) if '.csv' in f]:  
    try:
        fname = file.split('.')[0]
        df = pd.read_csv(path+file, sep=';', engine='python')
        df2 = df[df["&empty; Leistung (kW)"] > 0]
        lz = pd.to_timedelta(df2["Zeitspanne"]).sum()
        lz = lz.total_seconds()
        kwmin = df2["&empty; Leistung (kW)"].min()
        kwmax = df2["&empty; Leistung (kW)"].max()
        kwmean = df2["&empty; Leistung (kW)"].mean()
        data = {'kW_min':[kwmin],'kW_max':[kwmax], 'kW_mean':[kwmean], 'Ladezeit':[lz]}
        df = pd.DataFrame(data)
        daten.append(df.assign(ID=fname))
    except:
        pass

df = pd.concat(daten)
df.to_csv("Leistung_Ladezeit.csv", sep=";", index=False)


In [ ]:
## Zusammenfassen der Ladevoränge von WF und SZ.

df1 = pd.read_csv(r'Datenpfad\Ladevorgänge_WF_H.csv', sep=";")
df2 = pd.read_csv(r'Datenpfad\Ladevorgänge_WF_R.csv', sep=";")
df3 = pd.read_csv(r'Datenpfad\Ladevorgänge_SZ.csv', sep=";")
concatenated = pd.concat([df1, df2, df3])
concatenated.to_csv("Ladevorgänge_H_R_SZ.csv", sep=";", index=False)

In [ ]:
## Zusammenfassen der Ladevorgänge mit den Leistungsgrößen und der Ladezeit.  

df1 = pd.read_csv(r'Datenpfad\Ladevorgänge_H_R_SZ.csv', sep=";")
df2 = pd.read_csv(r'Datenpfad\Leistung_Ladezeit.csv', sep=";")
df3 = pd.merge(df1, df2, on='ID')
df3.to_csv("Rohdaten_Ladesäulen.csv", sep=";", index=False)